<a href="https://colab.research.google.com/github/sneha-4515/IoT-Predictive-Maintenance-Engine/blob/main/Notebooks/oT_Predictive_Maintenance_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''import pandas as pd
import numpy as np

# Load data
telemetry = pd.read_csv("data/telemetry.csv", parse_dates=["datetime"])
failures = pd.read_csv("data/failures.csv", parse_dates=["datetime"])
machines = pd.read_csv("data/machines.csv")

# Sort values
telemetry = telemetry.sort_values(["machineID", "datetime"])

# Merge machine info
df = telemetry.merge(machines, on="machineID", how="left")

# Basic cleaning
df = df.drop_duplicates()
df = df.fillna(method='ffill')'''

In [1]:
!gdown 1EEbHCVVvjBRl0MNn7c7mOCoQ6LGINqR9

Downloading...
From: https://drive.google.com/uc?id=1EEbHCVVvjBRl0MNn7c7mOCoQ6LGINqR9
To: /content/PdM_telemetry.csv
100% 80.1M/80.1M [00:00<00:00, 169MB/s]


In [4]:
import pandas as pd
import numpy as np

# Load telemetry data
telemetry = pd.read_csv("PdM_telemetry.csv", parse_dates=["datetime"])

# Sort by machine and time (CRITICAL for time series)
telemetry = telemetry.sort_values(["machineID", "datetime"])

print(telemetry.head())


             datetime  machineID        volt      rotate    pressure  \
0 2015-01-01 06:00:00          1  176.217853  418.504078  113.077935   
1 2015-01-01 07:00:00          1  162.879223  402.747490   95.460525   
2 2015-01-01 08:00:00          1  170.989902  527.349825   75.237905   
3 2015-01-01 09:00:00          1  162.462833  346.149335  109.248561   
4 2015-01-01 10:00:00          1  157.610021  435.376873  111.886648   

   vibration  
0  45.087686  
1  43.413973  
2  34.178847  
3  41.122144  
4  25.990511  
datetime     0
machineID    0
volt         0
rotate       0
pressure     0
vibration    0
dtype: int64


In [13]:
telemetry.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 876100 entries, 0 to 876099
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   datetime   876100 non-null  datetime64[ns]
 1   machineID  876100 non-null  int64         
 2   volt       876100 non-null  float64       
 3   rotate     876100 non-null  float64       
 4   pressure   876100 non-null  float64       
 5   vibration  876100 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 40.1 MB


In [12]:
telemetry.describe()

,datetime,machineID,volt,rotate,pressure,vibration
count,876100,876100.000000,876100.000000,876100.000000,876100.000000,876100.000000
mean,2015-07-02 18:00:00,50.500000,170.777736,446.605119,100.858668,40.385007
min,2015-01-01 06:00:00,1.000000,97.333604,138.432075,51.237106,14.877054
25%,2015-04-02 12:00:00,25.750000,160.304927,412.305714,93.498181,36.777299
50%,2015-07-02 18:00:00,50.500000,170.607338,447.558150,100.425559,40.237247
75%,2015-10-02 00:00:00,75.250000,181.004493,482.176600,107.555231,43.784938
max,2016-01-01 06:00:00,100.000000,255.124717,695.020984,185.951998,76.791072
std,NaN,28.866087,15.509114,52.673886,11.048679,5.370361


In [7]:
print(telemetry.isnull().sum())

datetime     0
machineID    0
volt         0
rotate       0
pressure     0
vibration    0
dtype: int64


In [8]:
print(telemetry.duplicated().sum())

0


If dataset have missing value can use this code

In [10]:
'''
sensor_cols = ['volt', 'rotate', 'pressure', 'vibration']
telemetry[sensor_cols] = (
    telemetry
    .groupby("machineID")[sensor_cols]
    .transform(lambda x: x.interpolate(method='linear').ffill().bfill())
)'''

'\nsensor_cols = [\'volt\', \'rotate\', \'pressure\', \'vibration\']\ntelemetry[sensor_cols] = (\n    telemetry\n    .groupby("machineID")[sensor_cols]\n    .transform(lambda x: x.interpolate(method=\'linear\').ffill().bfill())\n)'

In [14]:
telemetry.groupby("machineID")["datetime"].diff().value_counts()

,count
datetime,
0 days 01:00:00,876000


Obs: Checking the time difference between consecutive readings is it is exactly 1 hour.

**LAG Features**

In [15]:
for col in sensor_cols:
    telemetry[f"{col}_lag1"] = (
        telemetry.groupby("machineID")[col].shift(1)
    )
    telemetry[f"{col}_lag2"] = (
        telemetry.groupby("machineID")[col].shift(2)
    )

In [17]:
telemetry.head()

,datetime,machineID,volt,rotate,pressure,vibration,volt_lag1,volt_lag2,rotate_lag1,rotate_lag2,pressure_lag1,pressure_lag2,vibration_lag1,vibration_lag2
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973,176.217853,NaN,418.504078,NaN,113.077935,NaN,45.087686,NaN
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847,162.879223,176.217853,402.747490,418.504078,95.460525,113.077935,43.413973,45.087686
3,2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144,170.989902,162.879223,527.349825,402.747490,75.237905,95.460525,34.178847,43.413973
4,2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511,162.462833,170.989902,346.149335,527.349825,109.248561,75.237905,41.122144,34.178847


Rolling **Statistics**

In [18]:
windows = [3, 6]  # 3-hour & 6-hour window

for col in sensor_cols:
    for w in windows:
        telemetry[f"{col}_roll_mean_{w}"] = (
            telemetry.groupby("machineID")[col]
            .rolling(window=w)
            .mean()
            .reset_index(0, drop=True)
        )

        telemetry[f"{col}_roll_std_{w}"] = (
            telemetry.groupby("machineID")[col]
            .rolling(window=w)
            .std()
            .reset_index(0, drop=True)
        )

# LAGS & Rolling create NANs

In [19]:
telemetry = telemetry.dropna()

Time based Split

In [20]:
# Example: use last 20% time as test
split_time = telemetry['datetime'].quantile(0.80)

train = telemetry[telemetry['datetime'] <= split_time]
test  = telemetry[telemetry['datetime'] > split_time]

print(train.shape, test.shape)

(700500, 30) (175100, 30)


In [23]:
'''features = telemetry.drop(columns=["datetime"])
features.to_csv("modeling_dataset_week1.csv", index=False)'''

In [27]:
! gdown 1EhOAKgE39ZPfySWR5ZxokI1TxOFOwlcV

Downloading...
From: https://drive.google.com/uc?id=1EhOAKgE39ZPfySWR5ZxokI1TxOFOwlcV
To: /content/PdM_failures.csv
100% 24.3k/24.3k [00:00<00:00, 48.9MB/s]


In [29]:
failures = pd.read_csv("PdM_failures.csv", parse_dates=["datetime"])

In [30]:
telemetry["failure_next_24h"] = 0

for machine in telemetry["machineID"].unique():
    machine_data = telemetry[telemetry["machineID"] == machine]
    failure_times = failures[failures["machineID"] == machine]["datetime"]

    for ft in failure_times:
        mask = (
            (telemetry["machineID"] == machine) &
            (telemetry["datetime"] >= ft - pd.Timedelta(hours=24)) &
            (telemetry["datetime"] < ft)
        )
        telemetry.loc[mask, "failure_next_24h"] = 1

/tmp/ipython-input-1857866762.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  telemetry["failure_next_24h"] = 0


In [31]:
X = telemetry.drop(columns=["failure_next_24h", "datetime"])
y = telemetry["failure_next_24h"]

In [33]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y   # Important for imbalanced data
)

In [34]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)

class_weights = dict(zip(classes, weights))
print("Class Weights:", class_weights)

Class Weights: {np.int64(0): np.float64(0.5099525486707367), np.int64(1): np.float64(25.61919391412479)}


In [36]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=10, n_estimators=200,
                       n_jobs=-1, random_state=42)

In [37]:
from sklearn.metrics import classification_report, confusion_matrix, f1_score, recall_score

rf_preds = rf_model.predict(X_test)

print("Random Forest Results\n")
print("F1 Score:", f1_score(y_test, rf_preds))
print("Recall:", recall_score(y_test, rf_preds))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, rf_preds))
print("\nClassification Report:\n", classification_report(y_test, rf_preds))

Random Forest Results

F1 Score: 0.2252588575051067
Recall: 0.9356348741954359

Confusion Matrix:
 [[149924  21778]
 [   220   3198]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.87      0.93    171702
           1       0.13      0.94      0.23      3418

    accuracy                           0.87    175120
   macro avg       0.56      0.90      0.58    175120
weighted avg       0.98      0.87      0.92    175120



In [38]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    scale_pos_weight = class_weights[1],  # Helps imbalance
    random_state=42
)

xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

In [39]:
xgb_preds = xgb_model.predict(X_test)

print("XGBoost Results\n")
print("F1 Score:", f1_score(y_test, xgb_preds))
print("Recall:", recall_score(y_test, xgb_preds))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, xgb_preds))
print("\nClassification Report:\n", classification_report(y_test, xgb_preds))

XGBoost Results

F1 Score: 0.2862151239226761
Recall: 0.8598595669982446

Confusion Matrix:
 [[157522  14180]
 [   479   2939]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.92      0.96    171702
           1       0.17      0.86      0.29      3418

    accuracy                           0.92    175120
   macro avg       0.58      0.89      0.62    175120
weighted avg       0.98      0.92      0.94    175120



In [40]:
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.7, 0.8, 1.0],
    'colsample_bytree': [0.7, 0.8, 1.0]
}

random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_grid,
    n_iter=15,
    scoring='f1',   # IMPORTANT: Not accuracy
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

best_xgb = random_search.best_estimator_

print("Best Parameters:", random_search.best_params_)

Fitting 3 folds for each of 15 candidates, totalling 45 fits
Best Parameters: {'subsample': 1.0, 'n_estimators': 200, 'max_depth': 10, 'learning_rate': 0.1, 'colsample_bytree': 1.0}


In [ ]:
best_preds = best_xgb.predict(X_test)

print("Tuned XGBoost Results\n")
print("F1 Score:", f1_score(y_test, best_preds))
print("Recall:", recall_score(y_test, best_preds))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, best_preds))
print("\nClassification Report:\n", classification_report(y_test, best_preds))